<a href="https://colab.research.google.com/github/cl-1/cl/blob/master/Machine_Learning_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import pandas as pd
from toolz import *
from toolz.curried import *

# paths to all the csvs files provided in the challenge.
trainPath   = "train.csv"
testPath    = "test.csv"
microPath   = "properties.csv"
macrosPaths = ["cpi.csv", "interest.csv", "rentIndex.csv", "vacant.csv"]
geoPath     = "geo_attributes.csv"

# load the csvs into pandas's Dataframe.
## load train&test_keys that contain unique identifier for each observation
train_keys = pd.read_csv(trainPath).assign(train = 1)
test_keys  = pd.read_csv(testPath).assign(train = 0).assign(price = pd.NA)
keys       = pd.concat([train_keys, test_keys])
keys       = keys.rename(columns = {"contractDate" : "date"})

## merge key with micro to borrow the date information in key.
_micro = pd.read_csv(microPath)
micro  = pd.merge(_micro, keys, on = "property_key")

## load all the macro files
macros = []
for macrosPath in macrosPaths:
    df = pd.read_csv(macrosPath)
    df = df.rename(columns = {"Data Series" : "date"})
    macros.append(df)

# load geo
geo = pd.read_csv(geoPath)

In [ ]:
def preprocess_micro(micro):

    def _floorRange(row, thresFloor = 41):

        floorRange = row["floorRange"]
        if floorRange != "-":
            floorMin = int(re.search("(\d+)-", floorRange).groups()[0])
            if floorMin >= thresFloor:
                row["floorRange"] = f"{thresFloor}-99"
        return row

    def _tenure(row):

        tenure = row["tenure"]
        if tenure == "Freehold" : row["tenure"] = 1
        else                    : row["tenure"] = 0
        return row

    micro = (micro
             .apply(_floorRange, axis = 1)
             .apply(_tenure, axis = 1))

    micro["date"] = pd.to_datetime(micro["date"])

    return micro


def preprocess_macros(cpi, interest, rentIndex, vacant):


    def yq2ym(df):


        # repeat the first row for desired interpolation result
        df = pd.concat([df.head(0), df])
        df.at[0, "date"] = "2023 1Q "

        df["date"] = (pd.to_datetime((df["date"]
                                      .str
                                      .replace(r"(\d+) (\d)Q ", r"\1-Q\2")))
                      .dt
                      .to_period('M'))
        df = df.set_index("date").resample("M", convention = "end").interpolate("linear")
        return df

    def ym2ym(df):

        df["date"] = (pd.to_datetime(df["date"], format = "%Y %b ")
                      .dt
                      .to_period('M'))
        return df

    def mergeDfs(dfs, on):

        # grab first dataframe
        all_merged = dfs[0]
        # loop through all but first data frame
        for to_merge in dfs[1:]:
            # result of merge replaces first or previously
            # merged data frame w/ all previous fields
            all_merged = pd.merge(all_merged, to_merge,
                                  how = 'inner',
                                  on  = on)
        return all_merged

    # apply appropriate date conversion function for each dataframe
    dfs = [ym2ym(cpi), ym2ym(interest), yq2ym(rentIndex), yq2ym(vacant)]

    # join all dataframs into one
    df = mergeDfs(dfs, on = "date")
    # convert peroid[M] to datetime
    df["date"] = df["date"].dt.to_timestamp()
    # shift month + 3
    df["date"] = df["date"] + pd.DateOffset(months=+3)

    return df

In [ ]:
micro = preprocess_micro(micro)
macro = preprocess_macros(*macros)

<ipython-input-63-2c301e00b591>:40: FutureWarning: The default value of regex will change from True to False in a future version.
  .replace(r"(\d+) (\d)Q ", r"\1-Q\2")))
<ipython-input-63-2c301e00b591>:40: FutureWarning: The default value of regex will change from True to False in a future version.
  .replace(r"(\d+) (\d)Q ", r"\1-Q\2")))


In [ ]:
_merged = pd.merge(micro, macro, on = "date", how = "left")
merged  = pd.merge(_merged, geo, on = ["street", "project", "district"], how = "left")
floorRange  = merged.fillna(merged.mean())

<ipython-input-65-4254b218f815>:3: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  floorRange  = merged.fillna(merged.mean())
<ipython-input-65-4254b218f815>:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  floorRange  = merged.fillna(merged.mean())


In [ ]:
merged['floorRange'].unique()

array(['01-05', '06-10', '16-20', '21-25', '26-30', '11-15', '-', '41-99',
       '36-40', '31-35', 'B1-B5'], dtype=object)

In [ ]:
# Define a function to extract the minimum and maximum floors from the range
def extract_min_max_floors(range):
    if range == '-':
        return pd.Series([None, None])
    elif range.startswith('B'):
        # Handle the "B" case
        start, end = range.replace('B', '').split('-')
        start = int(start)
        end = int(end)
        return pd.Series([start, end])
    else:
        # Handle the regular case
        start, end = range.split('-')
        return pd.Series([int(start), int(end)])

# Apply the function to the 'floorRange' column to extract the minimum and maximum floors
merged[['minFloor', 'maxFloor']] = merged['floorRange'].apply(extract_min_max_floors)

In [ ]:
merged = merged.drop(["floorRange"], axis=1)
merged[['minFloor', 'maxFloor']].head()

,minFloor,maxFloor
0,1.0,5.0
1,1.0,5.0
2,1.0,5.0
3,1.0,5.0
4,1.0,5.0


In [ ]:
merged = merged.fillna(0)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [ ]:
le = LabelEncoder()
merged['date'] = le.fit_transform(merged['date'])
merged['property_key'] = le.fit_transform(merged['property_key'])
merged['street'] = le.fit_transform(merged['street'])
merged['tenure'] = le.fit_transform(merged['tenure'])
merged['marketSegment'] = le.fit_transform(merged['marketSegment'])
merged['project'] = le.fit_transform(merged['project'])
merged['typeOfArea'] = le.fit_transform(merged['typeOfArea'])
merged['propertyType'] = le.fit_transform(merged['propertyType'])


In [ ]:
merged.columns

Index(['area', 'propertyType', 'district', 'typeOfArea', 'tenure', 'street',
       'project', 'marketSegment', 'property_key', 'date', 'price', 'train',
       'CPI', 'InterestRate', 'RentIndex', 'Available', 'Vacant', 'lat', 'lng',
       'num_schools_1km', 'num_supermarkets_500m', 'num_mrt_stations_500m',
       'minFloor', 'maxFloor'],
      dtype='object')

In [ ]:
train = merged[merged["train"] == 1]
X = merged.drop(["price"], axis=1)
y = merged["price"]

In [ ]:
forest = RandomForestRegressor()
forest.fit(X, y)

RandomForestRegressor()

In [ ]:
test  = merged[merged["train"] == 0]
test  = test.drop(["price"], axis=1)

In [ ]:
y_pred = forest.predict(test)

In [ ]:
_test  = pd.read_csv(testPath)

In [ ]:
_test['prediction'] = y_pred

In [ ]:
_test['prediction'] = y_pred

In [ ]:
_test.to_csv('test.csv', index=False)

In [ ]:
from google.colab import files

# Assuming you have a DataFrame named _test

# Save DataFrame as a CSV file
_test.to_csv('test.csv', index=False)

# Download the file to your local desktop
files.download('test.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>